# Part 2 - Consume messages from an Aiven for Apache Kafka®️ topic

In this notebook you will learn how to consume messages to an Apache Kafka topic

![Consume messages from an Apache Kafka Topic](../img/consume.png)

> [!NOTE]
>
> _If you haven't already created and setup your virtual environment, follow the steps from [0-setup.ipynb](0-setup.ipynb)_

## Create an Apache Kafka®️ consumer

The next step is to configure the Kafka consumer and have it run while we produce messages.

In [ ]:
# Import Python Modules
import json
import os

from confluent_kafka import DeserializingConsumer, KafkaError, KafkaException
from rich.console import Console
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

KAFKA_SERVICE_URI = os.getenv("KAFKA_SERVICE_URI")


def json_serializer(msg, s_obj):
    return json.loads(msg.decode('ascii'))

conf = {
    'bootstrap.servers': KAFKA_SERVICE_URI,
    'client.id': 'myclient',
    'group.id': 'pizzaioli',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    'key.deserializer': json_serializer
    }

consumer = DeserializingConsumer(conf)
console = Console()

finished = False
local_count = 0

try:
    consumer.subscribe(["pizzas"])
    with console.status("Please run the last Python block in 1-produce.ipynb"):
        while not finished:
            if (msg:=consumer.poll(timeout=1.0)) is None:
               continue
            elif msg.error():
                    raise KafkaException(msg.error())
            else:
                console.print(f"{msg.offset()}: {msg.key()}:{msg.value()}\n\n")
                local_count += 1
                finished = local_count == 2
finally:
    # Close down consumer to commit final offsets.
    consumer.close()    

### Good Job 💪

Things are picking up. Let's continue to [producing partitioned topics](3-produce-partitioned-topic.ipynb)